In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as mtp 
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss , acf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as best_arima_finder 

In [ ]:
data = pd.read_csv("AAPL.csv")
data2 = pd.read_csv("AAPL.csv")

In [ ]:
data.head(5)

In [ ]:
data.info

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
type(data.index)

In [ ]:
data.set_index('Date' , inplace=True)

In [ ]:
data.columns

#   Augmented Dickey-Fuller Test (ADF)

**The Augmented Dickey-Fuller Test (ADF)** is a unit root test commonly used to determine the stationarity of time series data. It is an extension of the original Dickey-Fuller test and is more powerful, capable of handling more complex models.

### Hypotheses:
- **Null Hypothesis (H0)**: The time series data is non-stationary.
- **Alternate Hypothesis (H1)**: The time series is stationary (or trend-stationary).

### **Test Procedure:**
The ADF test extends the Dickey-Fuller test equation by including a Close order regressive process in the model. This increases the thoroughness of the test while keeping the null hypothesis unchanged.

### **Interpretation:**
To reject the null hypothesis and conclude that the series is stationary, the p-value produced by the test should be less than the significance level (e.g., 0.05).

# Kwiatkowski Phillips Schmidt Shin (KPSS) Test

The Kwiatkowski Phillips Schmidt Shin (KPSS) test is another method used to determine the stationarity of time series data, specifically around a mean or linear trend.

## Hypotheses:
- **Null Hypothesis (H0):** The data is stationary.
- **Alternate Hypothesis (H1):** The data is not stationary.

### Test Procedure:

The KPSS test utilizes linear regression to divide a series into three components: a deterministic trend, a random walk, and a stationary error. It employs Ordinary Least Squares (OLS) regression to compute the equation.

### Interpretation:
For level stationarity, the intercept will have a fixed element, or the series will be stationary around a fixed level.

In [ ]:
data_test = data2[['Date', 'Close']].copy()
data_test.set_index('Date' , inplace=True)
data_test[:8]

In [ ]:
fig=mtp.figure(figsize=(15,6))
sns.lineplot(data=data_test,x='Date',y='Close')
mtp.tick_params(
    axis='x',        
    which='both',   
    bottom=False,      
    top=False,        
    labelbottom=False) 
mtp.show()

In [ ]:
adf = adfuller(data_test['Close'])
kps = kpss(data_test['Close'] , regression = "ct")

In [ ]:
print("Test Statistic " , adf[0])
print("p-value " , adf[1])
print("Critical Values")
for i,j in adf[4].items():
    print('\t%s: %.3f' %(i, j))

As the test statistic is greater than the critical value we accept the null hypotheses . the data is non stationary 


In [ ]:
print("Test Statistic " , kps[0])
print("p-value " , kps[1])
print("Critical Values")
for i,j in kps[3].items():
    print('\t%s: %.3f' %(i, j))

As the test statistics value is greater than the critical value, the null hypothesis is rejected. This indicates that the data is non-stationary.

In [ ]:
# Stationarising the non stationary data 
# we can do that by following methods 
# 1. Log transformation 
# 2. Square Root 
# 3. Cube Root 
# for this we will use the second method 

data_test_sqrt = np.sqrt(data_test['Close'])
data_test_diff = data_test_sqrt.diff().dropna()

In [ ]:
# now applying the tests again to the
# square_rooted data to check the stationarity of the dataset
adf2 = adfuller(data_test_diff)
kps2 = kpss(data_test_diff , regression="ct")

In [ ]:
print("Test Statistic " , adf2[0])
print("p-value " , adf2[1])
print("Critical Values")
for i,j in adf2[4].items():
    print('\t%s: %.3f' %(i, j))

As the ADF test statics is lesser (more negative) then the critical value becomes the reason to reject the null hypothesis. This indicates that the data is stationary.

In [ ]:
print("Test Statistic " , kps2[0])
print("p-value " , kps2[1])
print("Critical Values")
for i,j in kps2[3].items():
    print('\t%s: %.3f' %(i, j))

As the KPSS test statistics value is less than the critical value, the null hypothesis is not rejected. This indicates that the data is stationary

In [ ]:
mtp.figure(figsize=(15,8))
mtp.plot(data_test_diff,label="after")
mtp.plot(data_test['Close'],label="before")
mtp.tick_params(
    axis='x',        
    which='both',   
    bottom=False,      
    top=False,        
    labelbottom=False)
mtp.legend()
mtp.show()

Thus the Before line shows an increasing value of the Close stock per time wherase the After line shows a stationary value over time 

In [ ]:
mtp.plot(data_test_diff,label="after")

In [ ]:
mtp.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})

# Original Series
fig, axes = mtp.subplots(3, 2, sharex=True)
axes[0, 0].plot(data_test['Close']); axes[0, 0].set_title('Original Series')
plot_acf(data_test['Close'], ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(data_test_diff); axes[1, 0].set_title('1st Order Differencing')
plot_acf(data_test_diff, ax=axes[1, 1])

# 2nd Differencing
axes[2, 0].plot(data_test_sqrt.diff().diff()); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(data_test_sqrt.diff().diff().dropna(), ax=axes[2, 1])

mtp.show()

Partial autocorrelation can be imagined as the correlation between the series and its lag, after excluding the contributions from the intermediate lags. So, PACF sort of conveys the pure correlation between a lag and the series. This way, we will know if that lag is needed in the AR term or not.

𝑌𝑡=𝛼0+𝛼1𝑌𝑡−1+𝛼2𝑌𝑡−2+𝛼3𝑌𝑡−3

That is, suppose, if Y_t is the current series and Y_t-1 is the lag 1 of Y, then the partial autocorrelation of lag 3 (Y_t-3) is the coefficient  𝛼3  of Y_t-3 in the above equation.

In [ ]:
mtp.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})

fig, axes = mtp.subplots(1, 2, sharex=True)
axes[0].plot(data_test_diff); axes[0].set_title('1st Differencing')
axes[1].set(ylim=(0,5))
plot_pacf(data_test_diff, ax=axes[1])

mtp.show()

We can see that the Partical Auto-Correlation Lag is quite significant so we will take 
**p = 1**

In [ ]:
# The ACF tells how many MA terms are 
# required to remove any autocorrelation
# in the stationarized series.
# Therefore we will take the acf of the firt order differenciartion to find the q 

mtp.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})

fig, axes = mtp.subplots(1, 2, sharex=True)
axes[0].plot(data_test_diff); axes[0].set_title('1st Differencing')
axes[1].set(ylim=(0,4))
plot_acf(data_test_diff, ax=axes[1])

mtp.show()

As we can see only one lag above yhe significant line we take 1 as q 

In [ ]:
# there fore building ARIMA 1,1,1 Model

model = ARIMA(data_test['Close'] , order = (1,1,1))
model_fit = model.fit()
print(model_fit.summary())

In [ ]:
# Plotting Residual Errors 

residuals = pd.DataFrame(model_fit.resid)
fig, ax = mtp.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
mtp.show()

THe Density of is very fine with a mean of 0

In [ ]:
forecast = model_fit.forecast(steps=10)  # Adjust the number of steps as needed
mtp.plot(forecast, label='Forecast')
mtp.legend()
mtp.show()

In [ ]:
train = data_test.values[:8374]
test = data_test.values[8374:]

In [ ]:
model = ARIMA(train, order=(1, 1, 1))  
fitted = model.fit()  

forecast_results = fitted.get_forecast(steps=2094, alpha=0.05)  # 95% confidence interval

# Extract forecasted values, standard errors, and confidence intervals
fc = forecast_results.predicted_mean
se = forecast_results.se_mean
conf = forecast_results.conf_int()

# Make as pandas series
fc_series = pd.Series(fc)
lower_series = pd.Series(conf[:, 0])
upper_series = pd.Series(conf[:, 1])

# Plot
mtp.figure(figsize=(12,5), dpi=100)
mtp.plot(train, label='training')
mtp.plot(test, label='actual')
mtp.plot(fc_series, label='forecast')
mtp.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
mtp.title('Forecast vs Actuals')
mtp.legend(loc='upper left', fontsize=8)
mtp.show()

Metrics


In [ ]:
from sklearn.metrics import mean_absolute_percentage_error , mean_absolute_error ,mean_squared_error

def forecast_accuracy(forecast, actual):
    forecast = np.squeeze(forecast)  # Remove single-dimensional entries from the shape of an array
    actual = np.squeeze(actual)
    mape = mean_absolute_percentage_error(actual , forecast)
    me = np.mean(forecast - actual)             # ME
    mae = mean_absolute_error(actual,forecast)
    rmse = np.sqrt(mean_squared_error(actual, forecast))
    mpe = np.mean((forecast - actual)/actual)   # MPE
    mse = mean_squared_error(actual , forecast)
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'mse' : mse } )

forecast_accuracy(fc, test)

# Assuming fc and test are arrays containing the forecasted and actual values respectively
metrics = forecast_accuracy(fc, test)

# Print the metrics
for key, value in metrics.items():
    print(f'{key}: {value}')


Around 55.72% MAPE implies the model is about 44.23% accurate in predicting the next 2094 observations. Now we know how to build an ARIMA model manually.

In [ ]:
# using pmdarima for finding the best arima modeL
model_2 = best_arima_finder.auto_arima(
    data_test.values , 
    start_p= 1 , 
    start_q = 1 , 
    test= "kpss" , 
    max_p=3,
    max_q=3,
    d = None , 
    trace = True , 
    seasonal= False , 
    error_action= "ignore" , 
    suppress_warnings= True , 
    stepwise= True
)

In [ ]:
print(model_2.summary())

So as per the autoarima function we get p as 3 , q as 2 

In [ ]:
model_2.plot_diagnostics(figsize=(10,8))
mtp.show()

In [ ]:
n_periods = 24
fc, confint = model_2.predict(n_periods=n_periods, return_conf_int=True)
index_of_fc = np.arange(len(data_test.values), len(data_test.values)+n_periods)

# make series for plotting purpose
fc_series = pd.Series(fc, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)

# Plot
mtp.plot(data_test.values)
mtp.plot(fc_series, color='darkgreen')
mtp.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

mtp.title("Final Forecast of Usage")
mtp.show()

In [ ]:

data_test = data_test.reset_index(drop=True)
# Make predictions on the test data using the best ARIMA model
y_pred = model_2.predict(len(data_test))

# Calculate the metrics
mae = mean_absolute_error(data_test['Close'], y_pred)
mse = mean_squared_error(data_test['Close'], y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(data_test['Close'], y_pred)

# Print the metrics
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Percentage Error (MAPE):", mape)
